In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
import os
from six.moves import urllib
import zipfile
import tarfile
import urllib
import pandas as pd 
import gzip
#Import all the dependencies
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import os
from google.colab import drive
import copy
import numpy as np
import copy
from gensim.models.doc2vec import Doc2Vec

Read the file

In [ ]:
collection =  pd.read_csv('collection.tsv', sep='\t', header=None, names=[ 'pid', 'passage' ])
collection

In [ ]:
doc = collection
doc['passage'] = doc['passage'].str.lower()
doc['passage'] = doc['passage'].str.replace('[^a-zA-Z\s+]', '')
doc['passage'] = doc['passage'].str.replace('[\s+]', ' ')

Remove Stop Words

In [ ]:
# Removing Stop Words
# 33532
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# for i in range(len(doc['passage'] )):
#     doc['passage'][i] = [w for w in doc['passage'][i] if w not in stopwords.words('english')]
# # queries_eval['query'] = queries_eval.apply(lambda row: for w in row['query'] if w not in stopwords.words('english'), axis=1)

Training 

In [ ]:
import smart_open

def read_corpus(fname, tokens_only=False):
    # with smart_open.open(fname, encoding="utf-8") as f:
    for index, row in fname.iterrows():
        tokens = gensim.utils.simple_preprocess(row['passage'])
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [index])

# train_corpus = list(read_corpus(doc))

In [ ]:
  model= Doc2Vec.load("doc2v.model")
  train_corpus = list(read_corpus(doc))
  model.build_vocab(train_corpus, update=True)
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  print(len(model.wv.vocab)) 
  model.save("doc2v.model")
  print("Model Saved")
  #61113

Turn Passage into Vector

In [ ]:
# index
model= Doc2Vec.load("doc2v.model")
collection_vec = copy.deepcopy(collection)
collection_vec['passage'] = collection.apply(lambda row : list(model.infer_vector(gensim.utils.simple_preprocess(row['passage']))), axis=1)
file_name = "feature_data/collection_vector.tsv"
with open(file_name ,'w') as write_tsv:
    write_tsv.write(collection_vec.to_csv(sep='\t',index=False))


Round the vec

In [ ]:
def turn_to_round(passage):
  round_ = passage[1:-1].split(",")
  round_list=[]
  round_list=[ round(float(num), 2) for num in round_]
  return round_list

collection_vec_round = copy.deepcopy(collection)
collection_vec_round = collection_vec.apply(lambda row: turn_to_round(row['passage']), axis=1)
file_name = "feature_data/collection_vec_round.tsv"
with open(file_name ,'w') as write_tsv:
      write_tsv.write(collection_vec_round.to_csv(sep='\t',index=False))

Test

In [ ]:
import random
# Pick a random document from the test corpus and infer a vector from the model
model= Doc2Vec.load("doc2v.model")

doc = "manhattan project"
inferred_vector = model.infer_vector(gensim.utils.simple_preprocess(doc))
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document : «{}»\n'.format(doc))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))